In [109]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [110]:
test=pd.read_csv('test_AbJTz2l.csv')

In [111]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 488.3+ KB


In [112]:
test['Item_Weight'].fillna(test['Item_Weight'].median(),inplace=True)
test['Outlet_Size'].fillna('U',inplace=True)

In [113]:
test.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [114]:
numerical_features=[feature for feature in test.columns if test[feature].dtype!='O' and feature not in test['Outlet_Establishment_Year']]

In [115]:
numerical_features.remove('Outlet_Establishment_Year')

In [116]:
test['Item_Fat_Content'].replace(['low fat','LF'],'Low Fat',inplace=True)

In [117]:
test['Outlet_No']=test['Outlet_Identifier'].str[3:]

In [118]:
test['Item_Fat_Content'].replace('reg','Regular',inplace=True)

In [119]:
test['Years active']=2013-test['Outlet_Establishment_Year']

In [120]:
test.drop(['Outlet_Identifier'],axis=1,inplace=True)

In [121]:
med=test['Item_Visibility'].median()
test['Item_Visibility'].replace(0,med,inplace=True)

In [122]:
test['Item_Type_Combined'] = test['Item_Identifier'].apply(lambda x: x[0:2])
test['Item_Type_Combined'] = test['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})

In [123]:
test.loc[test['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"

In [124]:
test.drop(['Item_Type'],axis=1,inplace=True)

In [125]:
Outlet_size=pd.get_dummies(test['Outlet_Size'],drop_first=True)
Outlet_location=pd.get_dummies(test['Outlet_Location_Type'],drop_first=True)
Outlet_Type=pd.get_dummies(test['Outlet_Type'],drop_first=True)
Item_Fat_Content=pd.get_dummies(test['Item_Fat_Content'],drop_first=True)


In [126]:
test.drop(['Outlet_Size','Outlet_Location_Type','Outlet_Type','Item_Fat_Content','Outlet_Establishment_Year'],axis=1,inplace=True)

In [127]:
test=pd.concat([test,Outlet_size,Outlet_location,Outlet_Type,Item_Fat_Content],axis=1)

In [128]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
test['Item_Type_Combined']=le.fit_transform(test.Item_Type_Combined)

In [129]:
test.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_No,Years active,Item_Type_Combined,Medium,Small,U,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3,Non-Edible,Regular
0,FDW58,20.750,0.007565,107.8622,049,14,1,1,0,0,0,0,1,0,0,0,0
1,FDW14,8.300,0.038428,87.3198,017,6,1,0,0,1,1,0,1,0,0,0,1
2,NCN55,14.600,0.099575,241.7538,010,15,2,0,0,1,0,1,0,0,0,1,0
3,FDQ58,7.315,0.015388,155.0340,017,6,1,0,0,1,1,0,1,0,0,0,0
4,FDY38,12.500,0.118599,234.2300,027,28,1,1,0,0,0,1,0,0,1,0,1


In [130]:
log_features=[feature for feature in numerical_features if feature not in['Item_Identifier'] and test[feature].min()!=0]

In [131]:
for feature in log_features:
  test[feature]=np.log(test[feature])

In [132]:
features_scale=[feature for feature in test.columns if feature not in['Item_Identifier','Item_Outlet_Sales']]
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(test[features_scale])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [133]:
test.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_No,Years active,Item_Type_Combined,Medium,Small,U,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3,Non-Edible,Regular
0,FDW58,3.032546,-4.884245,4.680854,049,14,1,1,0,0,0,0,1,0,0,0,0
1,FDW14,2.116256,-3.258977,4.469577,017,6,1,0,0,1,1,0,1,0,0,0,1
2,NCN55,2.681022,-2.306845,5.487920,010,15,2,0,0,1,0,1,0,0,0,1,0
3,FDQ58,1.989927,-4.174142,5.043644,017,6,1,0,0,1,1,0,1,0,0,0,0
4,FDY38,2.525729,-2.132005,5.456304,027,28,1,1,0,0,0,1,0,0,1,0,1


In [134]:
test_final=pd.concat([test[['Item_Identifier']].reset_index(drop=True),
                       pd.DataFrame(scaler.transform(test[features_scale]),columns=features_scale)],axis=1)

In [135]:
test_final.to_csv('X_test.csv',index=False)